In [2]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
def preprocess_data(file_path, max_time_steps=109, sample_rate=22050, duration=3, n_mels=128):
    audio, _ = librosa.load(file_path, sr=sample_rate, duration=duration)

    # Extract Mel spectrogram using librosa
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    if mel_spectrogram.shape[1] < max_time_steps:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :max_time_steps]

    return mel_spectrogram

In [3]:
# Set your file paths and constants
TRAINING_LABEL = '/data/common_source/datasets/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'
TRAINING_DATA = '/data/common_source/datasets/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_train/flac'
VALIDATION_DATA = '/data/common_source/datasets/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_dev/flac'
VALIDATION_LABEL = '/data/common_source/datasets/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trn.txt'
TEST_LABEL = '/data/common_source/datasets/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.test.trn.txt'
TEST_DATA = '/data/common_source/datasets/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_test/flac'
SAMPLE_RATE = 22050  # Adjust if your sample rate is different
DURATION = 3  # Adjust the duration of your audio samples
N_MELS = 128  # Adjust the number of mel filters

In [ ]:
# Load labels for training Data
labels = {}

with open(TRAINING_LABEL, 'r') as label_file:
    lines = label_file.readlines()

for line in lines:
    parts = line.strip().split()
    file_name = parts[1]
    label = 1 if parts[-1] == "bonafide" else 0
    labels[file_name] = label

X_train = []
y_train = []

max_time_steps = 109  # Define the maximum time steps for your model

for file_name, label in labels.items():
    file_path = os.path.join(TRAINING_DATA, file_name + ".flac")

    # Load audio file using librosa
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    # Extract Mel spectrogram using librosa
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    if mel_spectrogram.shape[1] < max_time_steps:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :max_time_steps]

    X_train.append(mel_spectrogram)
    y_train.append(label)

X_train = np.array(X)
y_train = np.array(y)

In [ ]:
# Load labels for evaluation Data
labels = {}

with open(VALIDATION_LABEL, 'r') as label_file:
    lines = label_file.readlines()

for line in lines:
    parts = line.strip().split()
    file_name = parts[1]
    label = 1 if parts[-1] == "bonafide" else 0
    labels[file_name] = label

X_dev = []
y_dev = []

max_time_steps = 109  # Define the maximum time steps for your model

for file_name, label in labels.items():
    file_path = os.path.join(VALIDATION_DATA, file_name + ".flac")

    # Load audio file using librosa
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    # Extract Mel spectrogram using librosa
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    if mel_spectrogram.shape[1] < max_time_steps:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :max_time_steps]

    X_dev.append(mel_spectrogram)
    y_dev.append(label)

X_dev = np.array(X_dev)
y_dev = np.array(y_dev)

In [ ]:
# Load labels for testing Data
labels = {}

with open(TEST_LABEL, 'r') as label_file:
    lines = label_file.readlines()

for line in lines:
    parts = line.strip().split()
    file_name = parts[1]
    label = 1 if parts[-1] == "bonafide" else 0
    labels[file_name] = label

X_test = []
y_test = []

max_time_steps = 109  # Define the maximum time steps for your model

for file_name, label in labels.items():
    file_path = os.path.join(TEST_DATA, file_name + ".flac")

    # Load audio file using librosa
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    # Extract Mel spectrogram using librosa
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    if mel_spectrogram.shape[1] < max_time_steps:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :max_time_steps]

    X_test.append(mel_spectrogram)
    y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [4]:
# Reshape input data to match the required input shape for ResNet
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))

# Reshape input data to match the required input shape for ResNet
X_dev = X_dev.reshape((X_dev.shape[0], X_dev.shape[1], X_dev.shape[2], 1))

# Define input shape and number of classes
input_shape = X_train[0].shape
num_classes = 2  # Assuming you have two classes (0 and 1)

In [ ]:
#Over sample
from imblearn.over_sampling import SMOTE

print(X_train.shape)
# Flatten the X_train data if needed
X_train_flattened = X_train.reshape(X_train.shape[0], -1)

# Instantiate the SMOTE object
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to the dataset
X_resampled, y_resampled = smote.fit_resample(X_train_flattened, y_train)

print(X_resampled.shape)

# Reshape X_resampled to match the shape of the original X_train
X_resampled_reshaped = X_resampled.reshape(-1, 13, 109)


print(X_resampled_reshaped.shape)

In [5]:
# Define the ResNet block
def resnet_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False):
    shortcut = x
    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

# Build the ResNet model
def build_resnet(input_shape, num_classes):
    input_tensor = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # ResNet blocks
    for size in [64, 128, 256, 512]:
        x = resnet_block(x, size, conv_shortcut=True)
        x = resnet_block(x, size)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=input_tensor, outputs=x, name='resnet_model')
    return model

In [6]:
# Build ResNet model
resnet_model = build_resnet(input_shape, num_classes)

2023-11-13 02:32:20.587112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 02:32:21.376269: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22834 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-11-13 02:32:21.376878: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22757 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:68:00.0, compute capability: 7.5


In [7]:
# Compile the model
resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Display the model summary
resnet_model.summary()

Model: "resnet_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 109, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 64, 55, 64)   3200        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 55, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [9]:
from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
epochs = 100  # Adjust the number of epochs as needed

resnet_model.fit(X_train, y_train, epochs=epochs, validation_data=(X_dev, y_dev))

# Evaluate the model on the test set
loss, accuracy = resnet_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10


2023-11-13 02:33:20.498318: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2023-11-13 02:33:21.778213: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


635/635 [==============================] - 63s 92ms/step - loss: 0.1717 - accuracy: 0.9307 - val_loss: 7.0544 - val_accuracy: 0.3349
Epoch 2/10
635/635 [==============================] - 58s 91ms/step - loss: 0.0806 - accuracy: 0.9695 - val_loss: 0.5567 - val_accuracy: 0.8989
Epoch 3/10
635/635 [==============================] - 58s 92ms/step - loss: 0.0522 - accuracy: 0.9814 - val_loss: 0.3153 - val_accuracy: 0.9427
Epoch 4/10
635/635 [==============================] - 58s 92ms/step - loss: 0.0306 - accuracy: 0.9901 - val_loss: 24.8457 - val_accuracy: 0.1675
Epoch 5/10
635/635 [==============================] - 59s 92ms/step - loss: 0.0332 - accuracy: 0.9891 - val_loss: 0.0201 - val_accuracy: 0.9933
Epoch 6/10
635/635 [==============================] - 59s 92ms/step - loss: 0.0199 - accuracy: 0.9938 - val_loss: 1.8837 - val_accuracy: 0.8599
Epoch 7/10
635/635 [==============================] - 59s 93ms/step - loss: 0.0184 - accuracy: 0.9943 - val_loss: 0.6738 - val_accuracy: 0.9005
Ep